In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image, ImageOps
import os
import json
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import torch.optim as optim

import cv2

In [2]:
# Dataset do složky DataSet -- Dataset1
#                           |- Dataset2

# Rozdělit Dataset na 2-3 Train, Test, Validation 
# Validation je až poslední
# shuffle data 
# balance data nemít 40% stejnou ground truth (tohle je asi u classifierů)

#### DEFINE DataLoader ####

class data(Dataset):

    def __init__(self, path):
        self.path = path  # check if exists
        self.files = os.listdir(path)

    # the function returns length of data
    def __len__(self):
        return len(self.files)

    # gives one item at a time
    def __getitem__(self, index):
        # OPEN JSON -> LOAD IMAGE PATHS  | probably set to grayscale
        #           -> LOAD GROUND TRUTH | probably change format
        filename = self.files[index]
        DatasetItem = json.load(open(os.path.join(self.path, filename)))

        convert_tensor = transforms.ToTensor()


        src = cv2.imread(DatasetItem['src'], cv2.IMREAD_GRAYSCALE)
        src = cv2.resize(src, (240, 135))
        src = convert_tensor(src) 

        dst = cv2.imread(DatasetItem['dst'], cv2.IMREAD_GRAYSCALE)
        dst = cv2.resize(dst, (240, 135))
        dst = convert_tensor(dst) 

        groundTruth = torch.tensor(DatasetItem['transformation'])

        stacked = torch.stack((src, dst))
        return stacked, groundTruth


In [3]:
#### DEFINE Neural Network ####
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self._to_linear = None

        # 2 input image channel, 64 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.fc9 = nn.Linear(64800, 1024)

        self.fc10 = nn.Linear(1024, 6)

    def convs(self, x):
        # max pooling over 2x2
        # 240 x 135
        # two conv layers
        x = self.conv1(x)
        x = self.conv2(x)

        # max pooling
        x = F.max_pool2d(x, 2)

        # 120 x 
        # two conv layers
        x = self.conv3(x)
        x = self.conv4(x)

        # max pooling
        x = F.max_pool2d(x, 2)

        # 60 x 
        # two conv layers
        x = self.conv5(x)
        x = self.conv6(x)

        # max pooling
        x = F.max_pool2d(x, 2)

        # 30 x 
        # two conv layers
        x = self.conv7(x)
        x = self.conv8(x)
            
        return x

    def forward(self, x):

        self.convs(x)
        x = x.view(-1, x[0].shape[0] * x[0].shape[1] * x[0].shape[2])

        # two linear layers
        x = self.fc9(x)
        x = self.fc10(x)
        
        return x


In [4]:
#### Create data ####

Dataset = data('.\\..\\Data\MachineData')
# TODO split data set (better)
train_set, val_set = torch.utils.data.random_split(Dataset, [154,30])

net = Net()

# Parameters
train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
test_dataloader = DataLoader(val_set, batch_size=3, shuffle=True)

for batch in train_dataloader:
    Images, GTtransformations = batch
    output = net(Images.view(-1, 2, 240, 135))
    print(output[0])
    break





# loss_function = nn.CrossEntropyLoss() # TODO better loss
# optimizer = optim.Adam(net.parameters(), lr=0.001)

# Epochs = 3
# # Loop over epochs
# for epoch in range(Epochs):
#     for data in train_dataloader:
#         Images, GTtransformations = data
#         net.zero_grad()  
#         output = net(Images.view(-1,1920, 1080)) 
#         loss = F.nll_loss(output, GTtransformations) # TODO 
#         loss.backward() 
#         optimizer.step() 
#     # Training

tensor([-0.4362,  0.2902, -0.4116,  0.2534, -0.1030,  0.0130],
       grad_fn=<SelectBackward0>)
